In [9]:
import os
from os import path
entity_dir = r'../data/ENTITY SET'

In [14]:
filenames = [path.join(entity_dir, fname) for fname in os.listdir(entity_dir)]
all_entities = []
for fname in filenames:
    with open(fname) as infile:
        content = unicode(infile.read(), 'gbk')
        all_entities.extend([i for i in content.split('\n') if len(i.strip()) != 0])

In [24]:
print len(all_entities)
print ' '.join(all_entities[:100])

140971
戴骏 戴复东 戴恩 桩桩 艾宝俊 阿康 阿发 旦丁 单福 孔丹 桓范 董槐 丁帆 あき 宝咖咖 宝珣 北北 阿甲 丁姬 都欢 甲丁 姬发 方垕 董方卓 东方骏 蒋川 卓敬 艾敬 东靖川 戴都都 冀金亮 方芳(北京四中教师) 方炯镔 蒋建军(宜兴陶艺师) 蒋敬(古典小说《水浒传》中人物) 荆棘公主 江都公主(明懿文太子之女) 蒋经国 蒋建国(北京市人民政府参事) 季冠霖 蒋静(节目主持人) 蒋静(青岛大学教授) 季晨 方方(成都理工大学教授) 蒋建明(工艺美术师) 东郊(明朝御史) 蒋敬(穆斯林翻译家) 蒋建平(上海海事大学海洋环境与工程学院讲师) 蒋建平(河南农业大学原校长) 季婷 蒋方婷 江建明(上海市服饰学会会长) 蒋芳(阿里巴巴集团副总裁) 季诺维也夫 季米特洛夫(政治人物) 本间雅晴 江建民(台湾音乐制作人) 江姐(中国民族歌剧) 江建明(河北省卫生和计划生育委员会副主任) 季历 稻姬(历史人物) 季诺(阿根廷漫画大师) 蒋建琪 净界法师 江角真纪子 季风之隐 季芹 季小军 季克良 方芳(四川大学华西第二医院主任医师) 娇娇(歌手) 季仁葵 方芳(上海财经大学副教授) 急救员 方纪 方芳(武警成都军事法院院长) 季文美 冀文林 季堂锐太 季托 经纪人(职业) 冀涛 范晶晶(潍坊学院教师) 戴季陶 季羡林 季大纯 方骏(画家) 季鹰(晋朝文学家) 京极真 季益顺 蒋方智怡 蒋方震 京本有加 方芳(中国人民大学经济学院教授) 季颖(新民晚报副总编辑) 方方(原全国侨联副主席) 季允石 季颖(沈阳大学讲师) 季鹰(外科医生) 京本大我


In [19]:
sub_tasks = ['celebrity', 'movie', 'restaurant', 'tvShow']
trainset_locs = map(path.abspath, ['../data/TRAIN SET/%s.TRAINSET.txt' % t for t in sub_tasks])
devset_locs = map(path.abspath, ['../data/DEV SET/%s.DEVSET.txt' % t for t in sub_tasks])
output_locs = map(path.abspath, ['../output/%s.txt' % t for t in sub_tasks])
def LoadInData(data_loc, test_data=False):
    lines = unicode(open(data_loc).read(), 'gbk').split('\n')
    parsing_result = []
    for line in lines:
        terms = line.split('\t')
        items = []
        for i in terms[1:]:
            if test_data:
                ent, score = i, None

            else:
                colon_separated = i.split(':')
                ent = ':'.join(colon_separated[:-1])
                score = int(colon_separated[-1])
            items.append((ent, score))
        if len(items) == 0:
            continue
        parsing_result.append((terms[0], items))
    return parsing_result

train_data = map(LoadInData, trainset_locs)
dev_data = [LoadInData(d, True) for d in devset_locs]

In [22]:
used_in_cur_data = set()
for data_block in train_data + dev_data:
    for ent in data_block:
        for r, g in ent[1]:
            used_in_cur_data.add(r)

In [23]:
print len(used_in_cur_data)

96126


# Searching on Baidu


In [30]:
import urllib

request_template = 'http://baike.baidu.com/search?%s'

entity_name = all_entities[0]
#urllib.urlopen('')
args = urllib.urlencode({"word" : entity_name.encode('utf8'), 'pn' : 0, 'rn' : 0, 'enc' : 'utf8'})
request = request_template % args

response = urllib.urlopen(request).read()

In [61]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response)
for result in soup.find_all('dd'):
    try:
        title = result.find_all('a', {'class' : 'result-title'})[0]
        summary = result.find_all('p', {'class' : 'result-summary'})[0]
        print title.text.replace(u'_百度百科', ''), title['href'], summary.text
    except IndexError:
        continue

戴骏 http://baike.baidu.com/view/4180050.htm 戴骏，上海人，中国国家游泳队队员，参加过广州亚运会，伦敦奥运会。主项为400米，1500米。个人简介姓名：戴骏
性别：男
2010年广州亚运会中国...
2010年青年奥运会 http://baike.baidu.com/view/11752792.htm 2010年8月17日男子4x100米自由泳接力决赛中,由孙博威、戴骏、王曦明和何健彬组成的中国队最后时刻上演反超,最终以3分24秒46获得一枚银牌,俄罗斯队和南非队分获金...
新建二中北京校友会 http://baike.baidu.com/view/3624503.htm 北航校友会会长 戴骏飞 男 2009/9---2010/7 对外经贸校友会会长 钟颖 女 2009/9---2010/7 北林校友会会长 闵璐嘉 女 2009/9---2010/7 中国地质校友...
2012年全国游泳冠军赛 http://baike.baidu.com/view/8279911.htm 3 戴骏 上海队 15分09秒39 超A标 5最后一日编辑 4月7日,在男子4X100米混合泳接力决赛中,辜标荣与何健彬、周嘉威、曾宪梁组成的广东队以3分36秒86获得冠军,...
李昀琦 http://baike.baidu.com/view/4714631.htm 上午参加接力预赛的四个人当中,没有李昀琦,而是派出了蒋海琦、戴骏、蒋宇辉和张恩剑的组合。为什么这样临时换将?在现场观战的河南游泳中心主任杨青山道出缘由。“为了...
第十六届亚运会中国体育代表团 http://baike.baidu.com/view/4587954.htm 孙晓磊、张丰林、谢智、薛瑞鹏、李夏延、马祥、王帅、周嘉威、陈韦伍、吴鹏、陈寅、张子山、汪顺、王铖湘、黄朝升、陈祚、蒋海琦、刘俊午、李昀琦、蒋宇辉、戴骏和...
2010年新加坡青少年奥林匹克运动会 http://baike.baidu.com/view/3866419.htm 第2金 戴骏 男子自由泳400第3金 混4x100自 男女混合自由泳第4金 何健彬 男子仰泳100第5金 邓薇 举重58第6金 白安琪 女子仰泳200...
崔国伟 http://baike.baidu.com/view/6409335.htm 